In [ ]:
from tensorflow import keras
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.python.keras.preprocessing import image    
from tensorflow.python.keras.metrics import top_k_categorical_accuracy
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, BatchNormalization
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.optimizers import Adam, SGD
from tensorflow.python.keras.callbacks import ModelCheckpoint

import sklearn
from sklearn.datasets import load_files
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
from glob import glob
import random
from math import ceil
from os import listdir
from os.path import exists
import matplotlib.pyplot as plt                        
%matplotlib inline

import sys
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm import tqdm
from tensorflow.python.keras.callbacks import ModelCheckpoint

#DATASET = 'flowers'
DATASET = 'stanford'
# DATASET = 'dogs_v2'
AUGMENT_DATA = True
#MODEL_NAME = 'InceptionResNetV2'  not working, doesn't like maxpool
MODEL_NAME = 'InceptionV3'
EPOCHS = 5
BATCH_SIZE = 32
RUN_MODE = 'test2'

if MODEL_NAME == 'VGG16':
    from tensorflow.python.keras.applications.vgg16 import VGG16 as model_type, preprocess_input
elif MODEL_NAME == 'VGG19':
    from tensorflow.python.keras.applications.vgg19 import VGG19 as model_type, preprocess_input
elif MODEL_NAME == 'ResNet50':
    from tensorflow.python.keras.applications.resnet50 import ResNet50 as model_type, preprocess_input
elif MODEL_NAME == 'InceptionV3':
    from tensorflow.python.keras.applications.inception_v3 import InceptionV3 as model_type, preprocess_input
elif MODEL_NAME == 'Xception':
    from tensorflow.python.keras.applications.xception import Xception as model_type, preprocess_input
elif MODEL_NAME == 'InceptionResNetV2':
    from keras.applications.inception_resnet_v2 import InceptionResNetV2 as model_type, preprocess_input
# both mobilenets run but warn about input_shape not being square.  Don't trust them
elif MODEL_NAME == 'MobileNet':
    from tensorflow.python.keras.applications.mobilenet import MobileNet as model_type, preprocess_input
elif MODEL_NAME == 'MobileNetV2':
    from tensorflow.python.keras.applications.mobilenet_v2 import MobileNetV2 as model_type, preprocess_input
elif MODEL_NAME == 'DenseNet201':
    from tensorflow.python.keras.applications.densenet import DenseNet201 as model_type, preprocess_input
elif MODEL_NAME == 'NASNetLarge':
    from tensorflow.python.keras.applications.nasnet import NASNetLarge as model_type, preprocess_input


#print('tensorflow version:', tensorflow.__version__)
print('keras version:', keras.__version__)

# from os import environ
# if 'COLAB_GPU' in environ:
#     from google.colab import drive
#     drive.mount('/content/gdrive')
#     %cd '/content/gdrive/My Drive/work_dog93'

if DATASET == 'stanford':
    train_data_path = 'StanfordDogImages/train/'
    valid_data_path = 'StanfordDogImages/valid/'
    test_data_path = 'StanfordDogImages/test/'
elif DATASET == 'flowers':
    train_data_path = 'flower_split/train/'
    valid_data_path = 'flower_split/valid/'
    test_data_path = 'dogImages/test/'
else:
    print('************ incorrect dataset **********************')
    sys.exit()
    
print('Data: ', DATASET)
print('Augment Data: ', AUGMENT_DATA)
print('Model: ', MODEL_NAME)
print('Batch Size: ', BATCH_SIZE)

In [ ]:
# general utiity functions

def top2_acc(labels, logits): 
    return top_k_categorical_accuracy(y_true=labels, y_pred=logits, k=2)
    
def get_metrics():
    if DATASET == 'flowers':
        metrics = ['accuracy', top2_acc]
    else:
        metrics=['accuracy', 'top_k_categorical_accuracy']
    return metrics

def get_storage_file_names():
    base_name = DATASET + '_' + MODEL_NAME
    if AUGMENT_DATA:
        base_name += '_da'

    weight_file_name = 'saved_weights/weights_' + base_name + '.hdf5'
    model_file_name = 'saved_models/' + base_name + '_model.h5'
    
    return weight_file_name, model_file_name

In [ ]:
def make_scratch_data():
    if AUGMENT_DATA:
        scratch_generator = ImageDataGenerator(rescale=1/255,
                                                  rotation_range=10.0,
                                                  width_shift_range=0.1,
                                                  height_shift_range=0.1,
                                                  shear_range=0.05,
                                                  zoom_range=0.1,
                                                  horizontal_flip=True,
                                                  vertical_flip=False,
                                                  fill_mode="reflect")
    else:
        scratch_generator = ImageDataGenerator(rescale=1/255)

    scratch_train_batches = scratch_generator.flow_from_directory(train_data_path,
                                                       target_size=(350, 350),
                                                       batch_size=BATCH_SIZE,
                                                       shuffle=True,
                                                       class_mode='categorical')

    scratch_valid_batches = scratch_generator.flow_from_directory(valid_data_path,
                                                           target_size=(350, 350),
                                                           batch_size=BATCH_SIZE,
                                                           shuffle=True,
                                                           class_mode='categorical')
    return scratch_train_batches, scratch_valid_batches

In [ ]:
# The architecture is based off the VGG architecture, but with just 1 conv layer for each block.

def build_scratch_model(num_classes):
    model = Sequential()
    # Conv layer1
    model.add(Conv2D(32, 3, strides=(1,1), padding='same', activation='relu', input_shape=(350,350,3)))
    model.add(MaxPooling2D((2,2), strides= 2, padding='same'))

    # Conv layer2
    model.add(Conv2D(64, 3, strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPooling2D((2,2), strides= 2, padding='same'))

    # Conv layer3
    model.add(Conv2D(128, 3, strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPooling2D((2,2), strides= 2, padding='same'))

    # Conv layer4
    model.add(Conv2D(256, 3, strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPooling2D((2,2), strides= 2, padding='same'))

    # Conv layer5
    model.add(Conv2D(256, 3, strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPooling2D((2,2), strides= 2, padding='same'))

    #Flatten Layer
    model.add(GlobalAveragePooling2D())
    model.add(BatchNormalization())

    #Fully Connected Layer 2
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(), 
                  loss='categorical_crossentropy', 
                  metrics=get_metrics())
    
    return model

In [ ]:
def run_from_scratch():
    train_batches, valid_batches = make_scratch_data()   
    num_classes = len(set(train_batches.classes)) 

    train_steps = ceil(len(train_batches.filenames) / BATCH_SIZE)
    valid_steps = ceil(len(valid_batches.filenames) / BATCH_SIZE)

    weight_file_name, model_file_name = get_storage_file_names()
    checkpointer = ModelCheckpoint(filepath=weight_file_name, 
                                   monitor='val_acc',
                                   verbose=1, 
                                   save_best_only=True)

    model = build_scratch_model(num_classes)
    history = model.fit_generator(train_batches, 
                        steps_per_epoch = train_steps, 
                        epochs=EPOCHS,
                        callbacks=[checkpointer], 
                        verbose=1, 
                        validation_data=valid_batches,
                        validation_steps=valid_steps)
    print('first 25')
    print('\n', history.history)
    model.save(model_file_name)
    history = model.fit_generator(train_batches, 
                        steps_per_epoch = train_steps, 
                        epochs=EPOCHS,
                        callbacks=[checkpointer], 
                        verbose=1, 
                        validation_data=valid_batches,
                        validation_steps=valid_steps)
    print('2nd 25')

    print('\n', history.history)
    model.save(model_file_name)
    history = model.fit_generator(train_batches, 
                        steps_per_epoch = train_steps, 
                        epochs=EPOCHS,
                        callbacks=[checkpointer], 
                        verbose=1, 
                        validation_data=valid_batches,
                        validation_steps=valid_steps)
    print('3nd 25')
    print('\n', history.history)

    
    return history

if MODEL_NAME == 'scratch':
    history = run_from_scratch()
    print('\n\n************************************ done running')
    import sys
    sys.exit()

In [ ]:
def imagenet_prep(image_tensor):
    '''

        Takes the image tensors for one image. Applies the preprocessing need for the Resnet and VGG models.
        
        Inputs: Image tensor of shape (width, height, colorchannels)
        
        Returns: Tensor of same shape with color channels flipped and values centered around zero for 
        each color channel.
    '''
    # imagenet averages for RGB
    image_net_mean = np.array([103.939,116.779,123.68])
    image_tensor -= image_net_mean
    
    # Flip color channels
    image_tensor = image_tensor[:, :, ::-1]
    return image_tensor



In [ ]:
def get_batches_from_gen():
    if AUGMENT_DATA:
        generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                               rescale=1.,
                                               rotation_range=10.0,
                                               width_shift_range=0.1,
                                               height_shift_range=0.1,
                                               shear_range=0.05,
                                               zoom_range=0.1,
                                               horizontal_flip=True,
                                               vertical_flip=False,
                                               fill_mode="reflect")
    else:
        generator = ImageDataGenerator(preprocessing_function=preprocess_input)



    train_batches = generator.flow_from_directory(train_data_path,
                                                  target_size=(350, 350),
                                                  #target_size=(331, 331),
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=True,
                                                  class_mode='categorical')
 
    valid_batches = generator.flow_from_directory(valid_data_path,
                                                  target_size=(299, 299),
                                                  #target_size=(350, 350),
                                                  #target_size=(331, 331), #NAS
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=False,
                                                  class_mode='categorical')

    return train_batches, valid_batches

In [ ]:
def build_transfer_model(train_batches):
    num_classes = len(set(train_batches.classes))
    
    base_model = model_type(include_top=False, weights='imagenet')
    first_extra_layer = 1 + len(base_model.layers)
    #print('first extra layer', first_extra_layer)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)

    x = Dense(256, activation='relu')(x)   
    x = BatchNormalization()(x)

    preds = Dense(num_classes, activation='softmax')(x)   
    model = Model(inputs=base_model.input, outputs=preds)  
    
#     for i, layer in enumerate(model.layers):
#         print(i, layer)

    for layer in model.layers[:first_extra_layer]:
      layer.trainable=False
    for layer in model.layers[first_extra_layer:]:
      layer.trainable=True
    
    return model


In [ ]:
def run_pretrained_model(saved_weights_name, saved_model_name):
    train_batches, valid_batches = get_batches_from_gen()
    model = build_transfer_model(train_batches)

    model.compile(optimizer=Adam(), 
                  loss='categorical_crossentropy', 
                  metrics=get_metrics())
    
    checkpointer = ModelCheckpoint(filepath=saved_weights_name, 
                                   monitor='val_acc',
                                   verbose=1, 
                                   save_best_only=True)
    if RUN_MODE == 'train':
        results = model.fit_generator(generator=train_batches,
                   steps_per_epoch=ceil(train_batches.n/BATCH_SIZE),
                   validation_data=valid_batches,
                   validation_steps = ceil(valid_batches.n/BATCH_SIZE),
                   callbacks=[checkpointer], 
                   verbose=1,
                   epochs=EPOCHS)
    
        print(results.history)
        model.save(saved_model_name)
       
#         dependencies = {
#             'top2_acc': top2_acc
#         }

#         model = load_model('./saved_models/t50_stanford_InceptionV3_ad.h5',
#                           custom_objects=dependencies)
# #         model = load_model('./saved_models/stanford_InceptionV3_ad.h5')

    
# #         results = model.fit_generator(generator=train_batches,
# #                    steps_per_epoch=ceil(train_batches.n/BATCH_SIZE),
# #                    validation_data=valid_batches,
# #                    validation_steps = ceil(valid_batches.n/BATCH_SIZE),
# #                    callbacks=[checkpointer], 
# #                    verbose=1,
# #                    epochs=EPOCHS)
    
# #         print(results.history)
# #         model.save(saved_model_name)

    
    
# #         model = load_model('./saved_models/t50_flowers_Xception_ad.h5',
# #                           custom_objects=dependencies)
#           # 127 Xception?   295 Inception
#         for layer in model.layers[:295]:   
#             layer.trainable=False
#         for layer in model.layers[295:]:
#             layer.trainable=True
            
#         model.compile(optimizer=SGD(lr=0.0001, 
#                                     momentum=0.9),  
#                                     loss='categorical_crossentropy', 
#                                     metrics=get_metrics())

#         results = model.fit_generator(generator=train_batches,
#                    steps_per_epoch=ceil(train_batches.n/BATCH_SIZE),
#                    validation_data=valid_batches,
#                    validation_steps = ceil(valid_batches.n/BATCH_SIZE),
#                    callbacks=[checkpointer], 
#                    verbose=1,
#                    epochs=EPOCHS)
#         print(results.history)
#         model.save(saved_model_name)

        
    elif RUN_MODE == 'test':
        #model = load_model('./saved_models/flowers_InceptionV3_ad.h5')
        model.load_weights("./saved_weights/final_stanford_InceptionV3_ad_best.hdf5")
        score = model.evaluate_generator(generator=valid_batches,
                                     steps=ceil(valid_batches.n/BATCH_SIZE))
#         print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#         print(model.metrics_names)
        results = {}
        results[model.metrics_names[1]] = score[1]*100
        results[model.metrics_names[2]] = score[2]*100
        print('results: ', results)
        
    elif RUN_MODE == 'test2':
        
        def translate_to_imagenet_values(key):
            key = int(key)
            if key > 272:
                return key + 155 # dingo, dhole, African hunting dog
            else:
                return key + 151
            
        class_names = {}  
        for folder in listdir(valid_data_path):
            key, value = folder.split('.')
            key = translate_to_imagenet_values(key.strip())
            class_names[key] = value.strip()
            
        model = model_type(include_top=True, weights='imagenet')
        results = model.predict_generator(generator=valid_batches,
                                         steps=ceil(valid_batches.n/BATCH_SIZE))
        y_pred = np.argmax(results, axis = 1)   
        y_true = valid_batches.classes
        predict_count, mispredict_count, non_dog_count = 0, 0, 0
        
        for i in range(len(y_true)):
            predict_count += 1
            single_prediction = y_pred[i]
            single_actual = translate_to_imagenet_values(y_true[i])
            if single_prediction != single_actual:
                mispredict_count += 1
                if single_prediction not in class_names:
                    non_dog_count += 1
            
        print('accuracy: ', 1 - mispredict_count / predict_count)
        print('non-dog error percent', non_dog_count / mispredict_count)
        results = 1                   
    return results

In [ ]:
def full_run():
    base_name = DATASET + '_' + MODEL_NAME
    if AUGMENT_DATA:
        base_name = base_name + '_ad'
    result_file_name = base_name + '_results.h5'
    saved_weights_name = 'saved_weights/' + base_name + '_best.hdf5'
    saved_model_name = 'saved_models/' + base_name + '.h5'
    !touch result_file_name && rm result_file_name
    !touch saved_weights_name && rm saved_weights_name
  
    history = run_pretrained_model(saved_weights_name, saved_model_name)
#     print(history.history)
#     df = pd.DataFrame.from_dict(history.history)
#     df.to_hdf(result_file_name, 'df', format='t')
    
full_run()
print('\n\n************************************ done running')
import sys
sys.exit()